In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00


In [2]:
from pydrive2.auth import GoogleAuth
from google.colab import drive
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
import requests
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1Eqyq4awTT-ihwcOTKG80531UJqpl2vKQ' #<-- You add in here the id from you google drive file, you can find it

# Create a file handle and download the file
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data')  # Save the file as data

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_parquet('data')
column_names = [f'C{i}' for i in range(384)]
df.drop(column_names, inplace = True, axis = 1)

In [5]:
df.drop(['Opening', 'WhiteElo','BlackElo', 'Event'],inplace = True ,axis= 1)

In [6]:
df = df[df['Result']!='1/2-1/2']
class_mapping = {'0-1': 0, '1-0': 1}
df['Result'] = df['Result'].map(class_mapping)
df.rename(columns={'Result': 'label'}, inplace = True)
df.rename(columns={'Moves': 'text'}, inplace = True)
df.drop('ID', inplace = True, axis =1)

In [7]:
labels = df[['label']]
#df.drop('Result', inplace = True, axis = 1)

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(df, labels, test_size=0.2, random_state=42)

# Split the remaining 20% into 50% testing and 50% validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
import torch
#use this device to move your PyTorch tensors and models to the appropriate hardware.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Specifying the model name, I chose 'xlm-roberta-base' instead of 'xlm-roberta-large' due to times of execution and problems with the gpu memory
model_name = 'distilbert-base-uncased'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels = 1
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
X_test.head(5)

,label,text
18105,1,d4 f5 e3 Nf6 Nc3 g6 Bc4 Bg7 Nf3 d5 Bb3 a6 Ne5 ...
219019,0,d4 e6 c4 d6 Nf3 Bd7 b3 Be7 Bb2 c5 d5 e5 e3 Nf6...
5557,1,e4 e5 d4 d6 d5 Nf6 Nc3 b6 Bb5+ Bd7 Nf3 Bxb5 Nx...
157713,1,e4 e5 Nf3 Qf6 Nc3 c6 Bc4 Bc5 O-O d6
152632,0,e4 e5 Nf3 Nc6 Bc4 h6 c3 Nf6 O-O Bd6 Qb3 O-O a4...


In [11]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=256)

In [12]:
X_train = X_train.sample(frac = 0.1, random_state= 11)
#y_train = y_train.sample(frac = 0.5, random_state = 11)
X_test = X_test.sample(frac = 0.1, random_state= 11)
#y_test = y_test.sample(frac = 0.5, random_state = 11)
X_val = X_val.sample(frac = 0.1, random_state= 11)
#y_val = y_val.sample(frac = 0.5, random_state = 11)

In [13]:
del df

In [14]:
X_train.shape

(34786, 2)

In [15]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [16]:
X_test['label'] = X_test['label'].astype('float64')
X_val['label'] = X_val['label'].astype('float64')
X_train['label'] = X_train['label'].astype('float64')
from datasets import Dataset
#encoding the data with the tokenizer
X_test_set = Dataset.from_pandas(X_test, preserve_index = False)
X_train_set = Dataset.from_pandas(X_train, preserve_index = False)
X_val_set = Dataset.from_pandas(X_val, preserve_index = False)

In [17]:
test_token = X_test_set.map(tokenize, batched=True, batch_size=None)
valid_token = X_val_set.map(tokenize, batched=True, batch_size=None)
train_token = X_train_set.map(tokenize, batched=True, batch_size=None)

data_encoded = {"train" : train_token, "validation" : valid_token, "test" : test_token}

Map:   0%|          | 0/4348 [00:00<?, ? examples/s]

Map:   0%|          | 0/4348 [00:00<?, ? examples/s]

Map:   0%|          | 0/34786 [00:00<?, ? examples/s]

In [18]:
data_encoded['train'].features

{'label': Value(dtype='float64', id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [19]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments

#we define the metrics to evaluate or model
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [20]:
import os
#setting the place to save the results of the model.
if not os.path.isdir("results"):
   os.mkdir("results")

In [21]:
del y_test
del y_val
del y_train

In [22]:
batch_size = 16 #we cannot put more batches due to limit of gpu memory

#Training arguments of the model
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=1, #set 2 epochs due to really high times of execution with more epochs.
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="steps", #"epochs"
                                  fp16 = False, #speed optimization
                                  #save_strategy="no",
                                  gradient_accumulation_steps=8)

In [23]:
#set the trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_encoded["train"],
    eval_dataset=data_encoded["validation"],
    compute_metrics=compute_metrics,
)

In [24]:
del labels

In [ ]:
#fine tuning the model
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
#we save into google drive for executing the model in different sessions if we needed
import os
#save in a local folder (not drive)
if not os.path.isdir("model"):
   os.mkdir("model")

model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

#compressing the model
!tar -czf model_chess_prova.tgz model/

#acces to drive
from google.colab import drive
drive.mount('/content/drive')

!cp model_chess_prova.tgz "/content/drive/MyDrive/UJI/Big Data/CHESS?"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#we see the metrics in the test dataset
preds_output = trainer.predict(data_encoded["test"])
preds_output.metrics